In [1]:
import numpy as np
import pandas as pd
import re
import string
import nltk
from nltk.util import ngrams
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy 
import json
import random
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import os

In [5]:
def read_data(path): #input the path to the directory with data
    frames = []
    
    _, _, files = next(os.walk(path)) #create a list of all datafile names     
          
    for file in tqdm(files): #for every file in directory
        with open(path+"/"+file) as f: #read each file
            dataframe = pd.read_json(f) #convert file to dataframe
     
        frames.append(dataframe) #append each dataframe to list
    data = pd.concat(frames, sort=False) #make it one big dataframe
    
    return data, frames

In [55]:
data, frames = read_data("subset")

In [65]:
data.Domain.value_counts()

politiken.dk           64777
ekstrabladet.dk        63512
computerworld.dk       21450
berlingske.dk          15285
jv.dk                  11725
jyllands-posten.dk      9875
dr.dk                   7485
bt.dk                   6761
stiften.dk              6137
soundvenue.com          5971
altinget.dk             5476
sn.dk                   5400
ing.dk                  5250
gaffa.dk                5090
tv2.dk                  4905
finans.dk               4285
version2.dk             3741
journalisten.dk         3165
information.dk          2947
fyens.dk                2785
nordjyske.dk            1823
ekkofilm.dk             1660
seoghoer.dk             1575
tv2east.dk              1552
billedbladet.dk         1501
finanswatch.dk          1456
fagbladet3f.dk          1419
ejendomswatch.dk        1347
borsen.dk               1339
motormagasinet.dk       1328
helsingordagblad.dk     1240
fodevarewatch.dk        1138
jiyan.dk                1065
pov.international       1062
samvirke.dk   

In [74]:
computerworld = data[data.Domain == "computerworld.dk"]

In [75]:
computerworld

,level_0,index,Id,Domain,Body,Header,PublicationDate,Uri,Byline,TextHash
1822,1822,1828,840497,computerworld.dk,PROSA: ACTA gør os til zombier Den kontroversi...,PROSA: ACTA gør os til zombier -,2003-05-10T00:00:00,https://www.computerworld.dk/art/156434/prosa-...,Kasper Villum Jensen,-1479233348
1823,1823,1829,840496,computerworld.dk,Få Nyhedsbreve Gamle DR-ansatte starter net-ra...,Gamle DR-ansatte starter net-radio -,2002-05-08T00:00:00,https://www.computerworld.dk/art/137231/gamle-...,Kasper Villum Jensen,-977886698
1824,1824,1830,840495,computerworld.dk,DMI skrotter Google Maps efter nye krav fra Go...,DMI skrotter Google Maps efter nye krav fra Go...,2005-01-12T00:00:00,https://www.computerworld.dk/art/205490/dmi-sk...,Kasper Villum Jensen,-121235845
1825,1825,1831,840494,computerworld.dk,De danske domæne-regler strammes igen Efter so...,De danske domæne-regler strammes igen -,2002-10-07T00:00:00,https://www.computerworld.dk/art/138397/de-dan...,Kasper Villum Jensen,-939664427
1826,1826,1832,840493,computerworld.dk,Dansk fildelingstjeneste får del i Skype-milli...,Dansk fildelingstjeneste får del i Skype-milli...,2020-12-11T00:00:00,https://www.computerworld.dk/art/205005/dansk-...,Kasper Villum Jensen,1330698521
...,...,...,...,...,...,...,...,...,...,...
93854,108741,7999,484325,computerworld.dk,Direktør i den danske iværksættervirksomhed Ze...,Spilfirma: Stop klynkeriet og skaf investorer,2007-09-07T00:00:00,https://www.computerworld.dk/art/41277/spilfir...,Mads Elkær,0
93857,108744,8256,484068,computerworld.dk,Tilbage til hovedartiklen\nTilbage i 2002 kunn...,Send sms'er via computeren,2008-08-08T00:00:00,https://www.computerworld.dk/art/47290/send-sm...,Mads Elkær,0
93858,108746,8576,483748,computerworld.dk,"\nKasper Heine fortæller samtidig, at hvis dan...",Skader ikke filmbranchen,2024-04-08T00:00:00,https://www.computerworld.dk/art/45461/nye-bio...,Mads Elkær,0
93859,108747,8594,483730,computerworld.dk,Side 2 af 2 Fortsat fra: Her udvikler vi fremt...,Få adgang til det offentlige på én hjemmeside,2021-04-10T00:00:00,https://www.computerworld.dk/art/55856/her-udv...,Mads Elkær,0


In [76]:
for k in range(20): 
    print(computerworld["Body"].iloc[k])
    print("\n")

PROSA: ACTA gør os til zombier Den kontroversielle ACTA-aftale fik PROSA-medlemmer til at vandre rundt som zombier 1. maj. Se video-optagelser. 3. maj 2010 kl. 10.54En regnfuld lørdag blev Fælledparken fyldt med feststemte folk – og zombier. De zombie-klædte folk med blege ansigter var ikke socialister med for meget alkohol i blodet men derimod PROSA-medlemmer i forklædning. Fagforeningens unge medlemmer brugte igen i år arbejdernes internationale kampdag til at sætte fokus på en af foreningens mærkesager: Frihed og overvågning på nettet. Men hvad har zombier med it at gøre? ”Den handler om, at der i disse dage aftales rammerne for internettets fremtid i den såkaldte ACTA-aftale. En aftale, der kan få stor betydning for os som it-folk og som forbrugere. Vi mener, at der for tiden spreder sig en slags zombie-lignende virus i samfundet. Denne virus har nogle alvorlige følger. For eksempel forbruger vi løs uden at tage stilling til, hvilke rettigheder der er forbundet til vores forbrug. V

In [63]:
data.Domain == "bt.dk"

0        False
1        False
2        False
3        False
4        False
         ...  
99995    False
99996    False
99997    False
99998    False
99999    False
Name: Domain, Length: 276579, dtype: bool

In [37]:
top20_authors = data["Byline"].value_counts().index.tolist()[:20]

In [45]:
top20_author_df = data[data.Byline.isin(top20_authors)]

In [47]:
top20_author_df
del top20_author_df["index"]
del top20_author_df["TextHash"]
del top20_author_df["level_0"]

In [54]:
top20_author_df.Domain.value_counts()

computerworld.dk        17401
ekstrabladet.dk          8153
berlingske.dk            7740
jv.dk                    5907
politiken.dk             4263
bt.dk                    3842
finans.dk                2226
journalisten.dk          1700
stiften.dk               1619
finanswatch.dk           1456
ejendomswatch.dk         1347
seoghoer.dk              1153
fodevarewatch.dk         1138
mobilabonnement.dk        592
sn.dk                     402
tv2.dk                    392
techradar.com             210
jyllands-posten.dk         76
global.techradar.com       45
kapwatch.dk                27
amwatch.dk                 23
shippingwatch.dk           22
itwatch.dk                 20
medwatch.dk                15
energiwatch.dk              7
dpu.au.dk                   4
version2.dk                 4
advokatwatch.dk             4
ctwatch.dk                  2
femina.dk                   1
Name: Domain, dtype: int64